In [ ]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader

from torch.utils.data import DataLoader

train, test = dataset_loader((128, 128))

# todo change name to KFoldController
# I am using the KFoldController class to simply get 20% of the dataset as a validation fold. I won't do any
# k-Fold CV on the parameters search or else this will take forever
dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32)
validation_dataloader = DataLoader(dataset=validation, batch_size=32)

In [ ]:
import keras
from models.naive_dnn_gen.naive_dnn import NaiveDnnContainer
import keras_tuner

hyperparameters = keras_tuner.HyperParameters()
hypermodel = NaiveDnnContainer((3, 128, 128))
tuner = keras_tuner.Hyperband(
    hypermodel,
    objective='val_loss',
    directory="naive_dnn_search",
    executions_per_trial=10,  # should be highest possible
)

tuner.search_space_summary()
tuner.search(train_dataloader, epochs=100, validation_data=validation_dataloader,
             callbacks=[
                 # keras.callbacks.TensorBoard("/tmp/tb_logs"),
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
             ])

In [ ]:
models = tuner.get_best_models(num_models=18)
best_model = models[0]
best_model.summary()

# todo: Forse per il momento mi conviene fissare lr e altgri parametri di learning per poi riprenderli